In [14]:
# scripts/daily_trade_suggestions.py

import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import json
from pathlib import Path

DATA_RECOMENDATIONS = Path("data/recommendations")
DATA_SUGGESTIONS = Path("data/suggestions")

for p in (DATA_RECOMENDATIONS, DATA_SUGGESTIONS):
    p.mkdir(parents=True, exist_ok=True)


# directory where CSVs live
DATA_DIR = Path("data")

# all generated daily files (ignore OUTCOME)
daily_files = [
    f for f in DATA_RECOMENDATIONS.glob("trade_suggestions_*.csv")
    if "OUTCOME" not in f.name
]

print(f"Found {len(daily_files)} daily files")

 


Found 117 daily files


In [15]:
dfs = [pd.read_csv(f) for f in daily_files]
df_all = pd.concat(dfs, ignore_index=True)
df_all.head()

,Unnamed: 0,price,ma50,mean_ret_fwd_1d,mean_ret_fwd_5d,mean_ret_fwd_20d,appearances,appearances_persistent,avg_HotScore,rule_score
0,RGC,47.519901,18.948798,5.610254,26.650771,111.145055,24.0,301.0,0.849029,6
1,SNDK,389.284485,242.671091,1.867555,13.692754,26.302457,18.0,253.0,0.822619,6
2,MU,345.859009,256.593182,1.820570,8.827528,32.169058,20.0,267.0,0.791470,6
3,ASTS,99.160400,71.092308,1.665916,8.771584,24.967006,22.0,345.0,0.760582,6
4,VERA,47.544998,39.516900,0.446763,8.120328,30.793858,16.0,248.0,0.823859,6


In [16]:
df_all = df_all.rename(columns={"Unnamed: 0": "ticker"})
df_all.head()

,ticker,price,ma50,mean_ret_fwd_1d,mean_ret_fwd_5d,mean_ret_fwd_20d,appearances,appearances_persistent,avg_HotScore,rule_score
0,RGC,47.519901,18.948798,5.610254,26.650771,111.145055,24.0,301.0,0.849029,6
1,SNDK,389.284485,242.671091,1.867555,13.692754,26.302457,18.0,253.0,0.822619,6
2,MU,345.859009,256.593182,1.820570,8.827528,32.169058,20.0,267.0,0.791470,6
3,ASTS,99.160400,71.092308,1.665916,8.771584,24.967006,22.0,345.0,0.760582,6
4,VERA,47.544998,39.516900,0.446763,8.120328,30.793858,16.0,248.0,0.823859,6


In [17]:
agg_map = {
    "price": "mean",
    "ma50": "mean",
    "mean_ret_fwd_1d": "mean",
    "mean_ret_fwd_5d": "mean",
    "mean_ret_fwd_20d": "mean",
    "appearances": "sum",
    "appearances_persistent": "max",
    "avg_HotScore": "mean",
    "rule_score": "max"
}

df_outcome = (
    df_all
    .groupby("ticker", as_index=False)
    .agg(agg_map)
)

df_outcome.sort_values("avg_HotScore", ascending=False).head()

,ticker,price,ma50,mean_ret_fwd_1d,mean_ret_fwd_5d,mean_ret_fwd_20d,appearances,appearances_persistent,avg_HotScore,rule_score
331,RGC,31.128044,20.542077,3.431695,17.173441,94.598242,3005.0,329.0,0.849555,6
416,VERA,46.847829,41.670729,0.446763,7.664952,21.450928,1872.0,248.0,0.823859,6
361,SNDK,444.220096,265.861961,2.048670,13.220684,56.712541,2477.0,292.0,0.823007,6
238,LITE,347.583985,323.500581,0.346679,3.339552,9.330412,2274.0,287.0,0.796931,6
268,MU,373.350515,270.401322,2.020016,7.993496,36.565373,2734.0,315.0,0.796165,6


In [18]:
today = datetime.now().strftime("%Y%m%d")

out_file = DATA_SUGGESTIONS / f"trade_suggestions_{today}.csv"

df_outcome.to_csv(out_file, index=False)
